# MDF Single model learning

In [1]:
import pandas as pd
import numpy as np

In [2]:
from prepare_data import PROCESSED_BIG, PROCESSED_SUBSTANCES, TRAIN, TEST, SOURCE

-- get_unique_noconst --
- Number of found duplicated rows :  1389
- Remove constant columns :  (10846, 41) -> (10846, 41)
- Dropped const columns :  


In [3]:
PROCESSED_TRAIN = PROCESSED_BIG[PROCESSED_BIG[SOURCE] == TRAIN].drop(SOURCE, axis=1)
PROCESSED_TRAIN_SUBS = PROCESSED_SUBSTANCES[PROCESSED_SUBSTANCES[SOURCE] == TRAIN].drop(SOURCE, axis=1)

In [4]:
PROCESSED_TRAIN.columns.values

array([u'agrement col', u'date amm annee', u'date declar annee',
       u'etat commerc', u'forme pharma', u'libelle_ampoule',
       u'libelle_capsule', u'libelle_comprime', u'libelle_film',
       u'libelle_flacon', u'libelle_gelule', u'libelle_pilulier',
       u'libelle_plaquette', u'libelle_poche', u'libelle_sachet',
       u'libelle_seringue', u'libelle_stylo', u'libelle_tube',
       u'nb_ampoule', u'nb_capsule', u'nb_comprime', u'nb_film',
       u'nb_flacon', u'nb_gelule', u'nb_ml', u'nb_pilulier',
       u'nb_plaquette', u'nb_poche', u'nb_sachet', u'nb_seringue',
       u'nb_stylo', u'nb_tube', u'prix', u'statut', u'statut admin',
       u'titulaires', u'tx rembours', u'type proc', 'logprix',
       u'auriculaire', u'buccogingivale', u'cutan\xe9e', u'dentaire',
       u'endocanalaire', u'endosinusale', u'endotrach\xe9obronchique',
       u'gastrique', u'gastro-ent\xe9rale', u'gingivale', u'infiltration',
       u'inhal\xe9e', u'intra cholangio-pancr\xe9atique',
       u'intra-

### Metric MAPE

In [5]:
# Mean Absolute Percentage Error
def mape_error(y_true, y_pred, **kwards): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

In [7]:
libelle_cols=[u'libelle_ampoule',
       u'libelle_capsule', u'libelle_comprime', u'libelle_film',
       u'libelle_flacon', u'libelle_gelule', u'libelle_pilulier',
       u'libelle_plaquette', u'libelle_poche', u'libelle_sachet',
       u'libelle_seringue', u'libelle_stylo', u'libelle_tube']

In [8]:
df = PROCESSED_TRAIN

In [ ]:
X0 = df.drop(['prix', 'logprix'], axis=1)
#X0 = X0.drop(libelle_cols, axis=1)
y = df['prix'] 
ylog = df['logprix']

X = StandardScaler().fit_transform(X0)
X_train, X_test, y_train, y_test, ylog_train, ylog_test = train_test_split(X, y, ylog, train_size=0.75)

In [ ]:
print X.shape
pd.DataFrame(data=X).describe()

### Try out SVM on all columns without substances

In [ ]:
svr = SVR(kernel='linear', C=1)
svr.fit(X_train, ylog_train)
score = svr.score(X_train, ylog_train)
print score
res0 = svr.predict(X_train)
res1 = svr.predict(X_test)
print mape_error(ylog_train, res0)
print mape_error(ylog_test, res1)

In [ ]:
print ylog_test.tolist()[:10]
print res1[:10]

In [ ]:
svr = SVR(kernel='linear', C=1)
svr.fit(X_train, y_train)
score = svr.score(X_train, y_train)
print score
res0 = svr.predict(X_train)
res1 = svr.predict(X_test)
print mape_error(y_train, res0)
print mape_error(y_test, res1)

In [ ]:
print y_test.tolist()[:10]
print res1[:10]

Cross validation scoring

In [86]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

In [ ]:
scores = cross_val_score(svr, X, ylog, cv=5, scoring=make_scorer(mape_error))
print scores      
print scores.mean(), scores.std()

### Try out RandomForest on all columns without substances

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor

In [ ]:
rf = RandomForestRegressor(n_estimators = 50, n_jobs = -1, verbose = 0)

In [ ]:
rf.fit(X_train, ylog_train)
score = rf.score(X_train, ylog_train)
print score
res0 = rf.predict(X_train)
res1 = rf.predict(X_test)
print mape_error(ylog_train, res0)
print mape_error(ylog_test, res1)

Cross validation scoring

In [ ]:
scores = cross_val_score(rf, X, ylog, cv=5, scoring=make_scorer(mape_error))
print scores      
print scores.mean(), scores.std()

### Try out GradientBoostingRegressor on all columns without substances

In [ ]:
gbr = GradientBoostingRegressor(verbose = 0, max_features='auto')

In [ ]:
gbr.fit(X_train, ylog_train)
score = rf.score(X_train, ylog_train)
print score
res0 = rf.predict(X_train)
res1 = rf.predict(X_test)
print mape_error(ylog_train, res0)
print mape_error(ylog_test, res1)

### Try out SVM / Random Forest on substances and its quantities

In [38]:
types = [u'nb_capsule', u'nb_comprime',  u'nb_gelule', u'nb_ml']
packages = [u'nb_ampoule', u'nb_film', u'nb_flacon', u'nb_pilulier', u'nb_plaquette', u'nb_poche', u'nb_sachet', u'nb_seringue', u'nb_stylo', u'nb_tube']
types_and_packages = list(types)
types_and_packages.extend(packages)


In [39]:
df = PROCESSED_TRAIN
df_subs = PROCESSED_TRAIN_SUBS.drop(['_ENCODED_'], axis=1)

In [40]:
types

[u'nb_capsule', u'nb_comprime', u'nb_gelule', u'nb_ml']

In [132]:
index = 3
print "Select type only : ", types[index]
mask = df[types[index]] > 0
print "Found lines (med has the type): ", len(df[mask])
indices = range(len(types))
indices.remove(index)
for i in indices:
    mask &= df[types[i]] < 1
    
print "Found lines (med has the type only) : ", len(df[mask])
df_logprices = df[mask]['logprix']

Select type only :  nb_ml
Found lines (med has the type):  1199
Found lines (med has the type only) :  1199


In [133]:
from common.preprocessing_helper import drop_const_cols

def create_df_subs_quants(df_subs, series_quants):
    cols = df_subs.columns
    df_subs_quants = pd.DataFrame(columns=cols)
    for c in cols:
        df_subs_quants[c] = df_subs[c] * series_quants
    return df_subs_quants

In [134]:
df_subs_quants = create_df_subs_quants(df_subs[mask], df[mask][types[index]])

In [135]:
df_subs_quants_ = drop_const_cols(df_subs_quants)
print "- Remove constant columns : ", df_subs_quants.shape, '->', df_subs_quants_.shape
print "- Dropped const columns : ", 
if len(df_subs_quants.columns) > len(df_subs_quants_.columns):
    print df_subs_quants.columns.difference(df_subs_quants_.columns).values
print ""
df_subs_quants = df_subs_quants_
del df_subs_quants_

- Remove constant columns :  (1199, 1766) -> (1199, 673)
- Dropped const columns :  [ u' - A/CALIFORNIA/7/2009 A/CALIFORNIA/7/2009 ANALOGUE ANTIGENES GRIPPE H1N1PDM09 SOUCHE SOUCHE SURFACE VIRUS X-181'
 u' - A/SOUTH A/SWITZERLAND/9715293/2013 ANALOGUE AUSTRALIA/55/2014 FRAGMENTE GRIPPE H3N2 INACTIVE IVR-175 SOUCHE SOUCHE VIRUS'
 u' - A/SWITZERLAND/9715293/2013 A/SWITZERLAND/9715293/2013 ANALOGUE ANTIGENES GRIPPE H3N2 NIB-88 SOUCHE SOUCHE SURFACE VIRUS'
 ..., u'ZOLMITRIPTAN' u'ZONISAMIDE' u'ZOPICLONE']



In [136]:
X0 = df_subs_quants
ylog = df_logprices

X = StandardScaler().fit_transform(X0)
X_train, X_test, ylog_train, ylog_test = train_test_split(X, ylog, train_size=0.75)

In [137]:
print X.shape
pd.DataFrame(data=X).describe()

(1199, 673)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672
count,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1199.000000,1199.000000,1199.000000,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1199.000000,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1199.000000,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1199.000000,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1199.000000,1199.000000,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1199.000000,1199.000000,1.199000e+03,1199.000000,1.199000e+03,1.199000e+03,1.199000e+03,1199.000000,1199.000000,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1199.000000,1.199000e+03,1.199000e+03,1199.000000,1.199000e+03,1.199000e+03,1199.000000,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e+03,1.199000e

In [140]:
svr = SVR(kernel='linear', C=1)
svr.fit(X_train, ylog_train)
score = svr.score(X_train, ylog_train)
print score
res0 = svr.predict(X_train)
res1 = svr.predict(X_test)
print mape_error(ylog_train, res0)
print mape_error(ylog_test, res1)

0.9020595353
54.2231209938
130.87357852


In [141]:
print ylog_test.tolist()[:10]
print res1[:10]

[1.9712993830601329, 2.5160822672564502, 4.762857461823685, 1.297463147413275, 1.2947271675944, 3.326473835466219, 1.4398351280479205, 1.3056264580524357, 1.7298840655099674, 1.7334238922150915]
[ 1.74698237  2.41623673  8.45265718  1.13643547  1.84788418  1.87478467
  1.87478467  1.85478338  1.87478467  2.15848276]


In [127]:
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor

In [142]:
rf = RandomForestRegressor(n_estimators = 50, n_jobs = -1, verbose = 0)

In [143]:
rf.fit(X_train, ylog_train)
score = rf.score(X_train, ylog_train)
print score
res0 = rf.predict(X_train)
res1 = rf.predict(X_test)
print mape_error(ylog_train, res0)
print mape_error(ylog_test, res1)

0.86015887003
35.5000642418
54.0762699314


Cross validation scoring

In [144]:
scores = cross_val_score(rf, X, ylog, cv=5, scoring=make_scorer(mape_error))
print scores      
print scores.mean(), scores.std()

[ 84.06748325  56.30992649  58.89403932  37.72402839  27.67600518]
52.9342965251 19.4191774598
